In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##Using Convolutions with Complex Images

前回は、Fashion MNISTデータセットを使って画像分類器をトレーニングしました。この場合、分類するものが28x28の画像の中央にありました。これを次のレベルへと発展させ、分類するものが画像のどこにでもあっても画像の特徴を認識するための訓練を行います。

与えられた画像に馬もしくは人間が含まれているかを分類する分類器を構築します。それを実現するためにネットワークを訓練します。


Fashion MNISTの場合、データはKeras経由でTensorFlowに組み込みました。今回はそのようなデータがないので、訓練を行う前にデータの処理を行う必要があります。

まずはデータをダウンロードしてみましょう。



In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

以下のpythonコードはpythonのOSライブラリを使用して、実行環境にあるファイルシステムへのアクセス権を与えます。そうすることで、zipfileライブラリを使用してファイルシステムに保存しているデータを解凍することができます。

In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

zipファイルの内容はベースディレクトリ `/tmp/horse-or-human` に展開され、 `horses` と `humans` というディレクトリが含まれています。

トレーニングセットは、ニューラルネットワークに「馬はこう見えます」「人間はこう見える」ということを教えるために使用されるデータです。

このサンプルで注意すべきことが一つあります。このサンプルでは、画像に馬か人間かを明示的にラベル付けされていません。先ほどのファッションの例では、「これは1です」「これは7です」などとラベル付けされていました。

*ImageGenerator* と呼ばれるものが使用されます。これは、サブディレクトリから画像を読み込んで、そのサブディレクトリの名前から自動的にラベルを付けるようにコード化されています。例えば、'training' ディレクトリに 'horses' ディレクトリと 'human' ディレクトリがあるとします。ImageGenerator が画像に適切なラベルを付けてくれるので、コーディングの手間が省けます。

それでは、それぞれのディレクトリを定義してみましょう。

In [0]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

では、`horses` と `humans` のトレーニングディレクトリのファイル名がどのようになっているか見てみよう。

In [0]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

馬と人のそれぞれの画像の数を調べてみましょう。

In [0]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))

では、いくつかの写真を見てみましょう。まず、matplotのパラメータを設定します。

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

8頭の馬と8人の人間の写真を表示します。セルを再実行すると、毎回新しい画像が表示されます。

In [0]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


## Building a Small Model from Scratch

続ける前にモデルの定義を始めましょう。

Step 1では、tensorflowをインポートします。

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  import tensorflow as tf
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
print(tf.__version__)

次に、先ほどの例と同様に畳み込みレイヤーを追加し、最終的な結果を平坦化して全結合層にフィードします。

最後に、全結合層を追加します。

我々は2クラス分類問題、すなわち*二項分類問題*に直面しているので、我々のネットワークは[*sigmoid*](https://wikipedia.org/wiki/Sigmoid_function)で終了することに注意してください。これにより、ネットワークの出力は0から1の間の1つのスカラー値となり、現在の画像がクラス1である確率を出力します（クラス0ではなく）．

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.summary()メソッドを呼び出すと、ニューラルネットワークのサマリーが表示されます。

In [0]:
model.summary()

"output shape"の列は、各レイヤーで特徴マップのサイズがどのように変化するかを示しています。畳み込みレイヤーはパディングのためにサイズを少し小さくし、各プーリングレイヤーは寸法を半分にしています。

次に、モデル学習のための仕様を設定します。これはバイナリ分類問題であり、最終的な活性化関数はシグモイドであるため、`binary_crossentropy`の損失でモデルを訓練します。学習率が `0.001` の `rmsprop` オプティマイザを使用します。学習中は、分類の精度を監視したいと思います。

**注**. この場合、[確率的勾配降下](https://developers.google.com/machine-learning/glossary/#SGD) (SGD)よりも[RMSprop最適化アルゴリズム](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp)を使用する方が、RMSpropが学習レートのチューニングを自動化してくれるので好ましいです（それ以外のオプティマイザ [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) や [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad) などのオプティマイザも、学習中に学習率を自動的に調整してくれるので、ここでも同様に機能します)。

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

### Data Preprocessing

ソースフォルダ内の画像を読み込んで `float32` のテンソルに変換し、（ラベルを付けて）ネットワークに送り込むデータ生成器をセットアップしましょう。学習用、テスト用にそれぞれ1つずつの生成器を用意します。生成器は、サイズ300x300の画像とそのラベル（バイナリ）のバッチを生成します。

すでにご存知かもしれませんが、ニューラルネットワークに入力されるデータは、通常、ネットワークで処理しやすいように何らかの方法で正規化されているはずです(生のピクセル・データをニューラルネットワークにそのまま投入するのは珍しいことです)。ここでは、ピクセル値を `[0, 1]` の範囲になるように正規化することで画像を前処理します(元のピクセルデータはすべての値が `[0, 255]` の範囲になります)。

Kerasでは、`keras.preprocessing.image.ImageDataGenerator`クラスで `rescale` パラメータを使用してこれを行います。この `ImageDataGenerator` クラスを使うと、`.flow(data, labels)` または `.flow_from_directory(directory)` を使って拡張画像バッチ（とそのラベル）のジェネレータをインスタンス化することができます。これらのジェネレータは、データジェネレータを入力として受け取るKerasモデルメソッド（`fit_generator`, `evaluate_generator`, `predict_generator`が該当します）で利用できます。

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


### Training

15エポックの訓練をしてみましょう -- これは実行に数分かかるかもしれません。

エポックごとの値に注意してください（小林注．それぞれの指標の上下動に注目してくださいという意味です）。

損失と精度は訓練の進捗状況を示す大きな指標です。これは訓練データの分類を推測し、既知のラベルと比較して結果を計算します。精度とは、正しく推測できたデータの割合を意味します。

In [0]:
history = model.fit(
      train_generator,
      validation_data = validation_generator,  
      epochs=15,
      steps_per_epoch=8,
      validation_steps=8,
      verbose=1)

###Running the Model

それでは、実際にモデルを使って予測を実行してみましょう。このコードでは、ファイルシステムから1つ以上のファイルを選択し、それらをアップロードして、モデルを通して実行し、オブジェクトが馬か人間かを表示します。

インターネットからファイルシステムに画像をダウンロードして試すことができます。

トレーニングの精度が99%を超えているにもかかわらず、ネットワークが多くのミスをしていることに注意してください。

これは**過学習**と呼ばれるものによるもので、ニューラルネットワークが非常に限られたデータで訓練されていることを意味します（各クラスの画像は500枚程度しかありません）。つまり、ニューラルネットワークはトレーニングデータと似ているデータについてはうまく分類することができますが、その中にないものの場合は失敗してしまうということです。

これは、訓練するデータが多ければ多いほど、最終的なネットワークが良くなることを意味します。

データが限られているにもかかわらず、トレーニングをより良いものにするために使用できるテクニックはたくさんあります。その中には画像拡張と呼ばれるものがあります。今回は範囲を超えているため説明を省略します。

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

### Visualizing Intermediate Representations

畳み込みネットワークが学習した特徴の状況を知るために、入力がどのように変換されていくかを視覚化してみましょう。

学習セットからランダムな画像を選び、各レイヤーからのアウトプットを作成します。それぞれの行にある画像は、その特徴マップにある特定のフィルターです。このセルを再実行して、様々な訓練画像の中間表現を生成します。

In [0]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

ご覧のように、画像の生のピクセルから、抽象的でコンパクトな表現へと変化していきます。出力層に近い表現は、その層のネットワークが何に注目しているかを表し、出力層に近づくにつれ注目している表現＝特徴量がどんどん簡単になり、注目すべき点が少なくなっていきます。これを「スパース」と呼びます。特徴量のスパース化は、ディープラーニングの重要な特徴です。

これらの特徴量は、画像の元のピクセルに関する情報よりも、画像のクラス分けに関する情報と捉えることができ、学習をすすめることで洗練されていきます。そのため、畳み込みネットワーク（または一般的なディープネットワーク）は、情報蒸留パイプラインと考えることができます。

## Clean Up

次の演習を実行する前に、次のセルを実行してカーネルを終了させ、メモリリソースを解放します。

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)